<a href="https://www.kaggle.com/code/sofiamatias/learning-equality-model-sm?scriptVersionId=117499836" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Kaggle Challenge - Learning Equality

https://www.kaggle.com/competitions/learning-equality-curriculum-recommendations/overview

## Goal of the Competition

The goal of this competition is to streamline the process of matching educational content to specific topics in a curriculum. You will develop an accurate and efficient model trained on a library of K-12 educational materials that have been organized into a variety of topic taxonomies. These materials are in diverse languages, and cover a wide range of topics, particularly in STEM (Science, Technology, Engineering, and Mathematics).

Your work will enable students and educators to more readily access relevant educational content to support and supplement learning.

## Submission File

For each **topic_id** in the test set, you must predict a space-delimited list of recommended **content_ids** for that topic. The file should contain a header and have the following format:

~~~
topic_id,content_ids
t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c_76231f9d0b5e
t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c_ebb7fdf10a7e
t_00069b63a70a,c_11a1dc0bfb99
...
~~~

## Scoring
Mean F2 score

## Evaluation Metric - Efficiency Scoring
We compute a submission's efficiency score by:

\begin{equation} \text{Efficiency} = \frac{1}{ \text{Benchmark} - \max\text{F2} }\text{F2} + \frac{1}{32400}\text{RuntimeSeconds} \end{equation}


where **F2** is the submission's score on the main competition metric, **Benchmark** is the score of the benchmark sample_submission.csv, **maxF2** is the maximum  of all submissions on the Private Leaderboard, and **RuntimeSeconds** is the number of seconds it takes for the submission to be evaluated. The objective is to minimize the efficiency score.

During the training period of the competition, you may see a leaderboard for the public test data in the following notebook, updated daily: Efficiency Leaderboard. After the competition ends, we will update this leaderboard with efficiency scores on the private data. During the training period, this leaderboard will show only the rank of each team, but not the complete score.

# How To Solve This Challenge

## Model Train

* Calculate embeddings: for topics (title, description) and contents (title, description and text), use SentenceTransformer. Split sentences by language.
* Use KNN model: train model with content embeddings and use topic embeddings to predict content matches, use k=10, 20, 30, 50
* Calculate F2 to choose best k.
* Submit predictions
* Set X_train, X_test, y_train, y_test: split by category (use 'aligned' for validation), set has_content = True for validation. Check dimensions
* Use all features for X, including topic_id and content_id, topic_title and content_title. Use the KNN predictions to "mount" X and y. Check dimensions.
* Use correlations to get y: if KNN topic-content match with correlations, y is 1, else is 0
* Use SVM RBF or KNN again for multiclass classification
* Use RandomizedSearch to get best hyperparameters. Use model score "recall". Split the dataset in languages for training: train model for each language individually
* Calculate F2 and see if we've got improvements from KNN.
* Submit predictions

## Submissions

* Use submission sample and get topics and contents
* Apply KNN model to get topic-content matches. Get contents per topic
* Filter "good matches" with second classification model
* Compare predicts with submission sample and calculate F2

## Efficiency

* Use KNN and 2nd model without GPU and check if it takes a long time to calculate sample submissions

# Data

## Imports

In [1]:
drive_path = '/kaggle/input/'
dataset_path = '/kaggle/input/learningequalityfiles/'
work_path = '/kaggle/working/'
model_path = '/kaggle/input/sentence-transformer-package/ST-all-MiniLM-L6-v2-trained/ST-all-MiniLM-L6-v2-trained'

In [2]:
import sys, os
sys.path.append("../input/sentence-transformer-package/sentence-transformers-2.2.2/sentence-transformers-2.2.2") 
import sentence_transformers

In [3]:
import pandas as pd
import numpy as np
import string
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import precision_score, recall_score, fbeta_score
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_validate

# Data Collection

## Load dataframes

In [4]:
def read_file_in_chunks(filename):
    list_chunks = []
    chunksize = 10000
    with pd.read_csv(filename, chunksize=chunksize, index_col=0) as reader:
        for chunk in reader:
            list_chunks.append(chunk)
    return pd.concat(list_chunks)

In [5]:
# loading dataframes from existing .csv files
drive_path = '/kaggle/input/'

print (f"\nLoading dataframes...")

df_content = pd.DataFrame([])
df_topics = pd.DataFrame([])
df_corr = pd.DataFrame([])
df_sample = pd.DataFrame([])
for path, subdirs, files in os.walk(drive_path):
    for file in files:
        if file.endswith(".csv") == True:
            filepath = os.path.join(path, file)
            df = read_file_in_chunks(filepath)
            if set(['copyright_holder', 'kind']).issubset(df.columns):
                df_content = df.fillna({"title": "", "description": ""})
                display(df_content)
                print (f"\nLoaded 'df_content' from {filepath}")
            elif set(['channel', 'parent']).issubset(df.columns):
                df_topics = df.fillna({"title": "", "description": ""})
                display(df_topics)
                print (f"\nLoaded 'df_topics' from {filepath}")
            elif ('content_ids' in df.columns) and ('sample' not in file):
                df_corr = df
                display(df_corr)
                print (f"\nLoaded 'df_corr' from {filepath}")
            if ('sample' in file):
                df_sample = df


Loading dataframes...


,title,description,channel,category,level,language,parent,has_content
id,,,,,,,,
t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True
t_000095e03056,Unit 3.3 Enlargements and Similarities,,b3f329,aligned,2,en,t_aa32fb6252dc,False
t_00068291e9a4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True
t_00069b63a70a,Transcripts,,6e3ba4,source,3,en,t_4054df11a74e,True
t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True
...,...,...,...,...,...,...,...,...
t_fffb0bf2801d,4.3 Graph of functions,,e77b55,aligned,4,en,t_676e6a1a4dc7,False
t_fffbe1d5d43c,Inscribed shapes problem solving,Use properties of inscribed angles to prove pr...,0c929f,source,4,sw,t_50145b9bab3f,True
t_fffe14f1be1e,Lección 7,,6e90a7,aligned,6,es,t_d448c707984d,True



Loaded 'df_topics' from /kaggle/input/learning-equality-curriculum-recommendations/topics.csv


,content_ids
topic_id,
t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
t_00069b63a70a,c_11a1dc0bfb99
t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4
...,...
t_fff830472691,c_61fb63326e5d c_8f224e321c87
t_fff9e5407d13,c_026db653a269 c_0fb048a6412c c_20de77522603 c...
t_fffbe1d5d43c,c_46f852a49c08 c_6659207b25d5



Loaded 'df_corr' from /kaggle/input/learning-equality-curriculum-recommendations/correlations.csv


,title,description,kind,text,language,copyright_holder,license
id,,,,,,,
c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,NaN,es,NaN,NaN
c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,NaN,it,NaN,NaN
c_0000ad142ddb,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,video,NaN,es,NaN,NaN
c_0000c03adc8d,Nado de aproximação,Neste vídeo você vai aprender o nado de aproxi...,document,\nNado de aproximação\nSaber nadar nas ondas ...,pt,Sikana Education,CC BY-NC-ND
c_00016694ea2a,geometry-m3-topic-a-overview.pdf,geometry-m3-topic-a-overview.pdf,document,Estándares Comunes del Estado de Nueva York\n\...,es,Engage NY,CC BY-NC-SA
...,...,...,...,...,...,...,...
c_fffcbdd4de8b,2. 12: Diffusion,,html5,What will eventually happen to these dyes?\n\n...,en,CSU and Merlot,CC BY-NC-SA
c_fffe15a2d069,Sommare facendo gruppi da 10,Sal somma 5+68 spezzando il 5 in un 2 e un 3.\n\n,video,NaN,it,NaN,NaN
c_fffed7b0d13a,Introdução à subtração,Sal fala sobre o que significa subtrair. Os ex...,video,NaN,pt,NaN,NaN



Loaded 'df_content' from /kaggle/input/learning-equality-curriculum-recommendations/content.csv


,content_ids
topic_id,
t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
t_00069b63a70a,c_11a1dc0bfb99
t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4
...,...
t_fff830472691,c_61fb63326e5d c_8f224e321c87
t_fff9e5407d13,c_026db653a269 c_0fb048a6412c c_20de77522603 c...
t_fffbe1d5d43c,c_46f852a49c08 c_6659207b25d5



Loaded 'df_corr' from /kaggle/input/learningequalityfiles/correlations.csv


In [6]:
#Sampling 'df_topics' and 'df_content' with sample_submission
if ~df_sample.empty:
    df_sample = pd.read_csv(f'/kaggle/input/learning-equality-curriculum-recommendations/sample_submission.csv', index_col=0)
    df_topics = df_topics.join(df_sample, how='inner')
    df_sample = df_sample.reset_index()
    df_sample.content_ids = df_sample.content_ids.str.split(' ')
    df_sample = df_sample.explode('content_ids').set_index('content_ids')
    df_content = df_content.join(df_sample, how='inner')
    print (f"\nFiltered 'df_topics' and 'df_content' according to 'sample_submission'")
if df_content.empty and df_topics.empty:
    print ('Error: there are no topics or content files to work with')
    quit()


Filtered 'df_topics' and 'df_content' according to 'sample_submission'


In [7]:
# load 'correlations' data into pandas dataframe
# Restructure 'df_corr' (correlations): explode the target column to be more practical to use - one topic -> one content
y = df_corr.copy()
y['content_ids'] = y.content_ids.str.split(' ')
y = y.explode('content_ids')
y.reset_index(inplace=True)
print (f"\nChanged 'df_corr' to exploded 'y'")
display(y)


Changed 'df_corr' to exploded 'y'


,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d
1,t_00004da3a1b2,c_376c5a8eb028
2,t_00004da3a1b2,c_5bc0e1e2cba0
3,t_00004da3a1b2,c_76231f9d0b5e
4,t_00068291e9a4,c_639ea2ef9c95
...,...,...
279914,t_fff9e5407d13,c_d64037a72376
279915,t_fffbe1d5d43c,c_46f852a49c08
279916,t_fffbe1d5d43c,c_6659207b25d5
279917,t_fffe14f1be1e,c_cece166bad6a


## Create Topic Breadcrumbs

In [8]:
# define some helper functions and classes to aid with data traversal

def get_topic_breadcrumbs (df_topic):
    print (f"Creating breadcrumbs...")
    topic_breadcrumbs=[]
    topic_ids=[]
    if 'topic_id' == df_topics.index.name:
        col_name = 'topic_id'
    else:
        col_name = 'id'
    for i, topic in enumerate(df_topic.index):
        if (i % 5000 == 0) and (i > 0):
            print (f'Created {i} breadcrumbs...')
        if df_topic.loc[topic].parent in df_topic.index:
            topic_ids.append (topic)
            topic_breadcrumbs.append (Topic(topic).get_breadcrumbs(separator=' '))
    print (f"Creating dataframe from breadcrumbs 'df_topics_breadcrumbs'...")
    df_topics_breadcrumbs = pd.DataFrame (topic_breadcrumbs, 
                                          index = topic_ids,
                                          columns = ['topic_breadcrumbs'])
    return df_topics_breadcrumbs


def print_markdown(md):
    display(Markdown(md))

class Topic:
    def __init__(self, topic_id):
        self.id = topic_id

    @property
    def parent(self):
        parent_id = df_topics.loc[self.id].parent
        if pd.isna(parent_id):
            return None
        else:
            return Topic(parent_id)

    @property
    def ancestors(self):
        ancestors = []
        parent = self.parent
        while (parent is not None) and (parent.id in df_topics.index):
            ancestors.append(parent)
            parent = parent.parent
        return ancestors

    @property
    def siblings(self):
        if not self.parent:
            return []
        else:
            return [topic for topic in self.parent.children if topic != self]

    @property
    def content(self):
        if self.id in df_corr.index:
            return [ContentItem(content_id) for content_id in df_corr.loc[self.id].content_ids.split()]
        else:
            return tuple([]) if self.has_content else []

    def get_breadcrumbs(self, separator=" >> ", include_self=True, include_root=True):
        ancestors = self.ancestors
        if include_self:
            ancestors = [self] + ancestors
        if not include_root:
            ancestors = ancestors[:-1]
        return separator.join(reversed([a.title for a in ancestors]))

    @property
    def children(self):
        return [Topic(child_id) for child_id in df_topics[df_topics.parent == self.id].index]

    def subtree_markdown(self, depth=0):
        markdown = "  " * depth + "- " + self.title + "\n"
        for child in self.children:
            markdown += child.subtree_markdown(depth=depth + 1)
        for content in self.content:
            markdown += ("  " * (depth + 1) + "- " + "[" + content.kind.title() + "] " + content.title) + "\n"
        return markdown

    def __eq__(self, other):
        if not isinstance(other, Topic):
            return False
        return self.id == other.id

    def __getattr__(self, name):
        return df_topics.loc[self.id][name]

    def __str__(self):
        return self.title
    
    def __repr__(self):
        return f"<Topic(id={self.id}, title=\"{self.title}\")>"


class ContentItem:
    def __init__(self, content_id):
        self.id = content_id

    @property
    def topics(self):
        return [Topic(topic_id) for topic_id in df_topics.loc[df_corr[df_corr.content_ids.str.contains(self.id)].index].index]

    def __getattr__(self, name):
        return content_df.loc[self.id][name]

    def __str__(self):
        return self.title
    
    def __repr__(self):
        return f"<ContentItem(id={self.id}, title=\"{self.title}\")>"

    def __eq__(self, other):
        if not isinstance(other, ContentItem):
            return False
        return self.id == other.id

    def get_all_breadcrumbs(self, separator=" >> ", include_root=True):
        breadcrumbs = []
        for topic in self.topics:
            new_breadcrumb = topic.get_breadcrumbs(separator=separator, include_root=include_root)
            if new_breadcrumb:
                new_breadcrumb = new_breadcrumb + separator + self.title
            else:
                new_breadcrumb = self.title
            breadcrumbs.append(new_breadcrumb)
        return breadcrumbs

## Data Cleaning

## Data cleaning params

In [9]:
levels = {1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', 6: 'six', 7: 'seven', 8: 'eight',
          9: 'nine', 10: 'ten', 0: 'zero'}
topics_cols = ['title', 'description', 'topic_breadcrumbs']
content_cols = ['title', 'description', 'text']
cat_for_val = 'aligned'
print (f"\nLoaded cleaning parameters")


Loaded cleaning parameters


## Data cleaning functions

In [10]:
# Clean up text
def clean_text(text_col):
    """
    Clean ponctuation and special chars from a dataframe column
    """
    punctuations = string.punctuation
    text_col = text_col.str.replace('\W', ' ', regex=True)
    for punct in string.punctuation:
        text_col = text_col.str.replace(punct, ' ', regex=True)
    return text_col

## Topics

*   Changing 'level' values from integers to strings
*   Clean strings: ponctuation and special chars (\n\t...)
*   Sort values by language

In [11]:
def build_topic_features (topics_df, df_topics_breadcrumbs, levels, topic_cols):
    """
    Create 'topics_features' from df_topics, clean parameters and functions
    """
    print (f"\nCreating and cleaning topic features...")
    topics_features = topics_df.copy()
    topics_features = topics_features.replace ({'level': levels})
    topics_features = topics_features.merge (df_topic_breadcrumbs, how='outer', right_index=True, left_index=True)
    for col in topics_cols:
        topics_features[col] = clean_text(topics_features[col])
    topics_features['sentences'] = topics_features[topics_cols].apply(lambda x: '.'.join(x.dropna().astype(str)), axis=1)
    topics_features = topics_features.drop(columns=['parent'] + topics_cols) 
    print (f"\nCreated 'topic_features'")
    display (topics_features.head())
    
    return topics_features


## Contents


*   Remove columns 'copyright_holder' and 'language' (for 'language' assume that topic-content match with correlations is same language)
*   Clean strings: ponctuation and special chars (\n\t...)

In [12]:
def build_content_features (content_df, content_cols):
    """
    Create 'content_features' from df_content, clean parameters and functions
    """
    print (f"\nCreating and cleaning content features...")
    content_features = content_df.copy()
    for col in content_cols:
        content_features[col] = clean_text(content_features[col])
    content_features['sentences'] =  content_features[content_cols].apply(lambda x: '.'.join(x.dropna().astype(str)), axis=1)
    content_features = content_features.drop(columns=['copyright_holder'] + content_cols)
    print (f"\nCreated 'content_features'")
    display (content_features.head())
    
    return content_features

# Model training

### Models params

In [13]:
#General
device = 'cpu'
if torch.cuda.is_available():
    device = torch.device('cuda:0')
#Sentence Transformer
all_mini = 'all-MiniLM-L6-v2'
paraphrase = 'paraphrase-multilingual-MiniLM-L12-v2'
trained_model = model_path
# KNN
k = [5,10,20,30,50]
alg = 'kd_tree' #'ball_tree','brute'
leaf = 10
print (f"\nLoaded models parameters")


Loaded models parameters


### Scoring functions

In [14]:
def model_scoring (y_test, y_pred, average='macro'):
    """
    Calculate precision, recall and f2-score for y_test and y_pred
    """
    precision = precision_score(y_test['content_ids'], y_pred['content_ids'], average=average)
    print ('Precision:', precision)
    recall = recall_score(y_test['content_ids'], y_pred['content_ids'], average=average)
    print ('Recall:', recall)
    F2macro = fbeta_score(y_test['content_ids'], y_pred['content_ids'], beta=2, average='macro')
    print ('F2 macro:', F2macro)
    F2micro = fbeta_score(y_test['content_ids'], y_pred['content_ids'], beta=2, average='micro')
    print ('F2 micro:', F2micro)

In [15]:
def calculate_Fscore(prediction_df, actual_df):
    
    prediction_df['content_ids'] = prediction_df.content_ids.str.split(' ')
    prediction_df.columns=['topic_id', 'content_ids_pred']
    actual_df['content_ids'] = actual_df.content_ids.str.split(' ')
    actual_df.columns=['topic_id', 'content_ids_actual']
    df = pd.merge(prediction_df, actual_df, how='inner', on='topic_id')
    df['correct_pred'] = df[['content_ids_pred', 'content_ids_actual']].apply(lambda x: len([d for d in x[0] if d in x[1]]), axis=1)
    df['precision'] = df['correct_pred']/(df.content_ids_actual.str.len() + 1e-7)
    df['recall'] = df['correct_pred']/(df.content_ids_pred.str.len() + 1e-7)
    for beta in [0.5, 1, 2]:
        df['f'+str(beta)] = ((1 + beta**2) * df['precision'] * df['recall'])/((beta**2 * df['precision']) + df['recall'] + 1e-7) 
    
    return df

In [16]:
#example_pred = [{ 'topic_id': 't1', 'content_ids': 'c1 c2 c3 c10 c12 c13 c14'}, { 'topic_id': 't2', 'content_ids': 'c1 c2 c3 c14'}]
#example_actual = [{ 'topic_id': 't1', 'content_ids': 'c1 c2 c3 c4 c5'}, { 'topic_id': 't2', 'content_ids': 'c2 c4 c5'}]
#calculate_Fscore(pd.DataFrame(example_pred), pd.DataFrame(example_actual))

## Get embeddings

* Use output as pytorch tensor and normalize tensor embeddings
* Convert to numpy array

In [17]:
def get_embeddings(topics_features, content_features):
    """
    Calculate embeddings for both topics and content
    From text in columns:
    topic_cols=['title','description', 'topic_breadcrumbs'] 
    content_cols=['title','description', 'text']
    """
    device = 'cpu'
    if torch.cuda.is_available():
        device = torch.device('cuda:0')

    encoder = SentenceTransformer(trained_model)
    topics_id_for_embeddings=[]
    topics_embeddings=[]
    content_ids_for_embeddings=[]
    content_embeddings=[]
    topics_aux = []
    content_aux = []

    topics_sentences = topics_features['sentences']
    print (f'\nGetting embeddings for {len(topics_sentences)} sentences from topics')
    # topics embeddings
    topics_aux = encoder.encode(topics_sentences, convert_to_tensor=True, show_progress_bar=False)
    topics_id_for_embeddings = topics_sentences.index
    topics_embeddings = util.normalize_embeddings(topics_aux.to(device)).detach().cpu().numpy()
    # content embeddings
    content_sentences = content_features['sentences']
    print (f'\nGetting embeddings for {len(content_sentences)} sentences from contents')
    content_ids_for_embeddings = content_sentences.index
    content_aux = encoder.encode(content_sentences, convert_to_tensor=True, show_progress_bar=True)
    content_embeddings = util.normalize_embeddings(content_aux.to(device)).detach().cpu().numpy()
    
    return topics_embeddings, content_embeddings, topics_id_for_embeddings, content_ids_for_embeddings

## Apply Nearest Neighbours

* Use unsupervised NN. Use GPU when available, otherwise use sklearn NN

### topic-content predictions functions (df's)

In [18]:
def get_preds(list_aux, topics_id_for_embeddings):
    """
    df_preds = Dataframe with topic_id's and KNN chosen content_ids
    """
    df = pd.DataFrame(zip (list_aux, topics_id_for_embeddings), columns=['content_ids', 'topic_id'])
    df = df.iloc[:,[1,0]]
    df.content_ids = df.content_ids.str.join(' ')
    df = df.fillna('')
    return df

In [19]:
def get_true(df_corr, df):
    """
    Ground truth
    df_true = Dataframe with same topic_id's from 'get_preds' function but content_id's taken from correlations, 
    for score calculations,
    """
    df_true = df_corr.copy().reset_index().merge (df, how='right', on='topic_id')[['topic_id', 'content_ids_x']].rename(columns={'content_ids_x': 'content_ids'})
    df_true = df_true.fillna('')
    return df_true

### Calculate k neighbors for contents per topic and F2 score

In [20]:
def k_neighbors (k, topics_embeddings, content_embeddings, topics_id_for_embeddings, content_ids_for_embeddings, df_corr):
    """
    Calculate k neighbors of contents_id's for each topic_id
    """
        
    device = 'cpu'
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    if device == 'cpu':
        from sklearn.neighbors import NearestNeighbors
    else:
        import cudf
        import cuml
        from cuml.neighbors import NearestNeighbors

    # Check if number of samples is below preset K value and reconfigure content_embeddings 
    topics_count = int(sum(x.size for sublist in topics_embeddings for x in sublist)/384)
    #Model
    print ('\n K value:', k)
    model = NearestNeighbors(n_neighbors=k,
                             n_jobs=-1)

    print ('\nFitting KNN model...')
    # Fit and Predictions
    print (f"\nGetting content_ids for {topics_count} topics... ")
    nbrs = model.fit(content_embeddings)
    _, indices = nbrs.kneighbors(topics_embeddings)
    content_id_list = np.array([content_ids_for_embeddings[val] for row in indices for val in row]).reshape(indices.shape)
    
    print ('\nCalculating scores...')
    #Define dataframes for predictions and ground truth
    df_preds = get_preds(content_id_list, topics_id_for_embeddings)
    df_true = get_true(df_corr, df_preds)
    fscore = calculate_Fscore(df_preds.copy(), df_true.copy())
    print ('\nTopics to match content:', topics_count)
    print ('\nCorrect predictions:', fscore.correct_pred.sum())
    print ('\nF2 Score:', fscore.f2.mean())
    #print ('\n\nF2 score by sklearn functions:')
    #model_scoring (df_preds.copy(), df_true.copy(), average='micro')
    print ('\n')
    
    return model, df_preds, df_true, fscore

## K parameter vs F2 score

In [21]:
"""
Encoder = 'paraphrase'
Language_limit = 5
K0 = 5
F2_score0 = 0.14187983285626535
K1 = 10
F2_score1 = 0.14168308552036216
K2 = 20
F2_score2 = 0.13348882325195702
K3 = 30
F2_score3 = 0.1252348976238163
Language_limit = 20
K0 = 5
F2_score0 = 0.060680956642008434
k1 = 10
F2_score0 = 0.04720607798794654
k2 = 20
F2_score1 = 0.0348693757018806
k5 = 100
F2_score5 = 0.016168764671525976
Encoder = trained_model
Language_limit = 10
k1 = 2 #limited by the sample dataset
F2_score1 = 0.32051279617850204
"""


"\nEncoder = 'paraphrase'\nLanguage_limit = 5\nK0 = 5\nF2_score0 = 0.14187983285626535\nK1 = 10\nF2_score1 = 0.14168308552036216\nK2 = 20\nF2_score2 = 0.13348882325195702\nK3 = 30\nF2_score3 = 0.1252348976238163\nLanguage_limit = 20\nK0 = 5\nF2_score0 = 0.060680956642008434\nk1 = 10\nF2_score0 = 0.04720607798794654\nk2 = 20\nF2_score1 = 0.0348693757018806\nk5 = 100\nF2_score5 = 0.016168764671525976\nEncoder = trained_model\nLanguage_limit = 10\nk1 = 2 #limited by the sample dataset\nF2_score1 = 0.32051279617850204\n"

# Submissions



In [22]:
#Getting parameters
K = k[1]
df_content

,title,description,kind,text,language,copyright_holder,license,topic_id
c_0c6473c3480d,Чертане на показателни финкции,"Сал чертае y=-2*3ˣ+5, като използва интерактив...",video,"Казват ни: ""Използвай интерактивната графика п...",bg,Khan Academy,CC BY-NC-SA,t_0006d41a73a8
c_1108dd0c7a5d,Молив като резистор,"Моливът причинява промяна в отклонението, подо...",video,NaN,bg,NaN,NaN,t_00004da3a1b2
c_11a1dc0bfb99,Flow Charts: Logical..: Written Transcript of ...,"Written Transcript of this video lesson, Engli...",document,Español\t\n \nTítulo\t\n de\t\n la\t\n lec...,en,MIT Blossoms,CC BY-NC-SA,t_00069b63a70a
c_1c57a1316568,Графики на показателни функции (стар пример),Сал свързва следните четири функции към подход...,video,"Тук имаме четири графики и 4 израза, задаващи ...",bg,Khan Academy,CC BY-NC-SA,t_0006d41a73a8
c_3695c5dc1df6,Additional Resources for Flow Charts: Logical ...,Additional resources and links.,html5,This MIT BLOSSOMS video lesson focuses on the ...,en,MIT Blossoms,CC BY-NC-SA,t_4054df11a74e
c_376c5a8eb028,Да чуем променливото съпротивление,Тук чертаем линия на лист хартия и я използвам...,video,NaN,bg,NaN,NaN,t_00004da3a1b2
c_5bc0e1e2cba0,Променлив резистор (реостат) с графит от молив,Използваме сърцевината на молива (неговия граф...,video,NaN,bg,NaN,NaN,t_00004da3a1b2
c_5e375cf14c47,Трансформиране на експоненциални графики (прим...,"Като е дадена графиката на y=2ˣ, Сал чертае y=...",video,"Казват ни: ""Графиката на у, равно на 2 на степ...",bg,Khan Academy,CC BY-NC-SA,t_0006d41a73a8
c_639ea2ef9c95,Dados e resultados de funções: gráficos,Encontre todas as entradas que correspondem a ...,exercise,NaN,pt,NaN,NaN,t_00068291e9a4
c_76231f9d0b5e,Последователно свързване на галваничен елемент...,"Защо отклонението се променя, когато се свърже...",video,NaN,bg,NaN,NaN,t_00004da3a1b2


In [23]:
#Getting topics breadcrumbs or topics context
print ("\n* Getting 'topics_breadcrumbs' dataframe...\n")
df_topic_breadcrumbs = get_topic_breadcrumbs (df_topics)

#Cleaning data and getting topics and contents features
print ('\n* Calculating topics_features dataframe...\n')
topics_features = build_topic_features (df_topics, df_topic_breadcrumbs, levels, topics_cols)

print ('\n* Calculating content_features dataframe...\n')
contents_features = build_content_features (df_content, content_cols)


* Getting 'topics_breadcrumbs' dataframe...

Creating breadcrumbs...
Creating dataframe from breadcrumbs 'df_topics_breadcrumbs'...

* Calculating topics_features dataframe...


Creating and cleaning topic features...

Created 'topic_features'


,channel,category,level,language,has_content,content_ids,sentences
t_00004da3a1b2,000cf7,source,four,bg,True,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...,Откриването на резисторите.Изследване на матер...
t_00068291e9a4,8e286a,source,four,pt,True,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...,Entradas e saídas de uma função.Entenda um pou...
t_00069b63a70a,6e3ba4,source,three,en,True,c_11a1dc0bfb99,Transcripts..Flow Charts Logical Thinking Tr...
t_0006d41a73a8,000cf7,source,four,bg,True,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...,Графики на експоненциални функции Алгебра 2 н...
t_4054df11a74e,6e3ba4,source,two,en,True,c_3695c5dc1df6 c_f2d184a98231,Flow Charts Logical Thinking .This lesson is ...



* Calculating content_features dataframe...


Creating and cleaning content features...

Created 'content_features'


,kind,language,license,topic_id,sentences
c_0c6473c3480d,video,bg,CC BY-NC-SA,t_0006d41a73a8,Чертане на показателни финкции.Сал чертае y 2...
c_1108dd0c7a5d,video,bg,NaN,t_00004da3a1b2,Молив като резистор.Моливът причинява промяна ...
c_11a1dc0bfb99,document,en,CC BY-NC-SA,t_00069b63a70a,Flow Charts Logical Written Transcript of ...
c_1c57a1316568,video,bg,CC BY-NC-SA,t_0006d41a73a8,Графики на показателни функции стар пример .С...
c_3695c5dc1df6,html5,en,CC BY-NC-SA,t_4054df11a74e,Additional Resources for Flow Charts Logical ...


In [24]:
#Calculating embeddings from text
print ('\n* Calculating embeddings for both topics and contents text columns...\n')
topics_embeds, contents_embeds, topic_id_embeds, content_ids_embeds = get_embeddings(topics_features, contents_features)


* Calculating embeddings for both topics and contents text columns...


Getting embeddings for 5 sentences from topics

Getting embeddings for 16 sentences from contents


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
#Calculating unsupervised k neighbours contents, for topics
print ('\n* Calculating k neighnors predictions...\n')
model, preds, ground_truth, fscore = k_neighbors (K, topics_embeds, contents_embeds, topic_id_embeds, content_ids_embeds, df_corr)

print ('\n All finished!\n')
print ('\n*** Predictions ***\n')
display(preds)
print ('\n*** Score ***\n')
display(fscore)


* Calculating k neighnors predictions...


 K value: 10

Fitting KNN model...

Getting content_ids for 5 topics... 

Calculating scores...

Topics to match content: 5

Correct predictions: 16

F2 Score: 0.3649385719557225



 All finished!


*** Predictions ***



,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d c_5bc0e1e2cba0 c_76231f9d0b5e c...
1,t_00068291e9a4,c_ebb7fdf10a7e c_639ea2ef9c95 c_ac1672cdcd2c c...
2,t_00069b63a70a,c_3695c5dc1df6 c_11a1dc0bfb99 c_f2d184a98231 c...
3,t_0006d41a73a8,c_1c57a1316568 c_b972646631cb c_0c6473c3480d c...
4,t_4054df11a74e,c_f2d184a98231 c_3695c5dc1df6 c_11a1dc0bfb99 c...



*** Score ***



,topic_id,content_ids_pred,content_ids_actual,correct_pred,precision,recall,f0.5,f1,f2
0,t_00004da3a1b2,"[c_1108dd0c7a5d, c_5bc0e1e2cba0, c_76231f9d0b5...","[c_1108dd0c7a5d, c_376c5a8eb028, c_5bc0e1e2cba...",4,1.0,0.4,0.769231,0.571429,0.454545
1,t_00068291e9a4,"[c_ebb7fdf10a7e, c_639ea2ef9c95, c_ac1672cdcd2...","[c_639ea2ef9c95, c_89ce9367be10, c_ac1672cdcd2...",4,1.0,0.4,0.769231,0.571429,0.454545
2,t_00069b63a70a,"[c_3695c5dc1df6, c_11a1dc0bfb99, c_f2d184a9823...",[c_11a1dc0bfb99],1,1.0,0.1,0.357143,0.181818,0.121951
3,t_0006d41a73a8,"[c_1c57a1316568, c_b972646631cb, c_0c6473c3480...","[c_0c6473c3480d, c_1c57a1316568, c_5e375cf14c4...",5,1.0,0.5,0.833333,0.666667,0.555556
4,t_4054df11a74e,"[c_f2d184a98231, c_3695c5dc1df6, c_11a1dc0bfb9...","[c_3695c5dc1df6, c_f2d184a98231]",2,1.0,0.2,0.555555,0.333333,0.238095


In [26]:
#import pickle
#pickle.dump(model, open(f'ST-KNN-{k[1]}', 'wb'))
preds.to_csv('submission.csv', index=False)